<a href="https://colab.research.google.com/github/econb/datajam-2022/blob/main/dajam_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CRC 2022

**Temática**: Contenidos Audiovisuales  
**Enunciado del reto**: ¿Qué estrategias pueden ser propuestas para promover la elección informada de los usuarios de televisión abierta a partir del contenido presentado en las parrillas de programación?  
**Descripción del reto**: Proponer estrategias de visualización y comunicación de datos de los contenidos audiovisuales de la televisión abierta, buscando que los usuarios puedan tomar decisiones sobre los contenidos que desean consumir, tomando en consideración las similitudes o diferencias existentes entre las parrillas de programación presentadas.  
**Tipo de reto**: Exploratorio  
**Datos sugeridos**: Parrilla de programación televisión abierta

#LECTURA DE DATOS

In [57]:
import pandas as pd
parrilla_file = "/content/drive/MyDrive/Colab Notebooks/DAJAM/Parrilla_programacion_8.csv"
parrilla = pd.read_csv(parrilla_file, sep=';')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [58]:
#Cleaning
parrilla = parrilla.replace('ND', pd.NA)
parrilla = parrilla.dropna(axis=0, subset=['PROGRAMA'])
parrilla.loc[parrilla['TIPO'].isna(), 'TIPO'] = "";
parrilla.describe(include='all')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,EMPRESA,ID_EMPRESA,TIPO_OPERADOR,ANNO,TRIMESTRE,FECHA,HORA_INICIO,HORA_FIN,DURACION,PROGRAMA,CLASIFICACION,GENERO,TIPO,CLOSED_CAPTION,LENGUA_SENAS,SUBTITULADO,LENGUAS_NATIVAS
count,1139606,1139606.0,1139606,1.139606e+06,1.139606e+06,1139606,1139606,1139602,1139601,1139606,1133839,1132366,1139606,1130301,1080616,1070605,1074183
unique,40,45.0,6,NaN,NaN,1005,85217,85268,11496,65071,4,4,642,3,2,2,2
top,CANAL CAPITAL,830012587.0,TV Pública - Regional Abierta,NaN,NaN,2020-07-11,06:00:00,06:00:00,00:30:00,CABEZOTE CONECTADOS 2017,FAMILIAR,NO FICCIÓN,EDUCATIVO,NINGUNO,NO,NO,NO
freq,242422,284825.0,616872,NaN,NaN,1990,17594,14705,272827,13417,887042,656133,149309,544765,1005841,1000582,1070032
mean,NaN,NaN,NaN,2.020440e+03,2.544827e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,8.591265e-01,1.164454e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,2.019000e+03,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,2.020000e+03,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,2.020000e+03,3.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,2.021000e+03,4.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#NORMALIZACIÓN DE TEXTO


In [ ]:
import spacy 
!python -m spacy download es_core_news_sm
nlp = spacy.load('es_core_news_sm')

!pip install pyspellchecker
from spellchecker import SpellChecker
spell = SpellChecker(language='es')

from tqdm import tqdm

In [54]:
def spanishNormalization(s):
  s2 = []
  tokens = nlp(s) #tokenization
  for token in tokens:
    token = str(token)
    if spell.unknown([token]): #mispellings
      token = spell.correction(token)
    token = token.lower() #lowercasing
    if (token.isalnum() and 
        not (token=="rep" or token.startswith("repetic"))): #stopwords
      s2.append(token)
  s2 = (' '.join(s2)).strip()
  return(s2)

In [ ]:
programas = (parrilla['PROGRAMA'] + " " + parrilla['TIPO']).unique() 
ps2 = [spanishNormalization(p) for p in tqdm(programas[50000:60000])]

In [ ]:
psdf = pd.DataFrame({'programa':ps2})
psdf.to_csv("/content/drive/MyDrive/Colab Notebooks/DAJAM/prog50000_60000.csv")

#EMBEDDING

In [35]:
import pandas as pd
pg1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DAJAM/prog0_10000.csv")
pg2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DAJAM/prog10000_20000.csv")
pg3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DAJAM/prog20000_30000.csv")
pg4 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DAJAM/prog30000_40000.csv")
pg5 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DAJAM/prog40000_50000.csv")
pg55 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DAJAM/prog50000_55000.csv")
pg6 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DAJAM/prog55000_60000.csv")
pg65 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DAJAM/prog60000_65000.csv")
pg7 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DAJAM/prog65000_70000.csv")

pg = pd.concat([pg1,pg2,pg3,pg4,pg5,pg55,pg6,pg65,pg7])
programas = pg['programa'].unique() #56338

In [ ]:
#https://huggingface.co/hiiamsid/sentence_similarity_spanish_es It maps sentences & paragraphs to a 768 dimensional dense vector space
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
sen2vecmodel = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

In [38]:
corpus = programas
embeddings = sen2vecmodel.encode(sentences=corpus, convert_to_numpy=True, batch_size=100, show_progress_bar=True)
embeddings

Batches:   0%|          | 0/564 [00:00<?, ?it/s]

array([[-1.4659004 ,  0.19834721, -0.67712575, ..., -0.16485667,
         0.1511169 , -0.41872445],
       [-0.03774842,  0.37663573, -1.157205  , ..., -0.9343896 ,
        -0.47136888,  0.13609478],
       [-0.89883626,  0.17250611, -0.91031593, ..., -0.15895092,
        -1.5804238 , -0.964409  ],
       ...,
       [-1.234079  ,  0.48303884,  0.16866267, ..., -0.4266604 ,
         0.8692584 ,  0.61074436],
       [-0.77485627, -0.55490893, -0.31367198, ..., -0.21574183,
        -0.20873678, -0.06140732],
       [-0.18701115, -0.97176653, -0.51445764, ..., -0.31380072,
        -0.41664535, -0.9980982 ]], dtype=float32)

In [41]:
print(embeddings.shape)
(pd.DataFrame(embeddings)).to_csv("/content/drive/MyDrive/Colab Notebooks/DAJAM/embeddings.csv")

(56338, 768)


# ENTRENAMIENTO MAPA AUTOORGANIZADO (RED NEURONAL)

In [40]:
!pip install minisom
from minisom import MiniSom 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
som = MiniSom(50, 50, 768, sigma=5,
              learning_rate=0.5, neighborhood_function='gaussian', topology='rectangular')
som.pca_weights_init(embeddings)
print("Training...")

Training...


In [52]:
som.train_random(data=embeddings, num_iteration=len(embeddings), verbose=True)  # random training
print("\n...ready!")

 [ 56338 / 56338 ] 100% - 0:00:00 left 
 quantization error: 12.836207922934808

...ready!


In [77]:
import pickle
# saving the som in the file som.p
with open('/content/drive/MyDrive/Colab Notebooks/DAJAM/som.p', 'wb') as outfile:
    pickle.dump(som, outfile)
#with open('"/content/drive/MyDrive/Colab Notebooks/DAJAM/som.p', 'rb') as infile:
#    som = pickle.load(infile)

# VISUALIZACIÓN MAPA CON LOS PROGRAMAS DEL DÍA


In [ ]:
programasDiaDf = parrilla.loc[parrilla['FECHA']=="2022-03-31"]
programasDia = programasDiaDf['PROGRAMA']

#normalizar
programasDiaNorm = [spanishNormalization(p) for p in tqdm(programasDia)]
#embedding
programasDiaEmbd = sen2vecmodel.encode(sentences=programasDiaNorm, convert_to_numpy=True, batch_size=100, show_progress_bar=True)
#Obtener ubicación del programa en mapa
l = list()
for i in range(len(programasDia)):
  x, y = som.winner(programasDiaEmbd[i])
  label = programasDia.iloc[i]
  row = [x,y,label]
  l.append(row)
programasEnMapa = pd.DataFrame(l, columns=['x','y','Programa'])

In [ ]:
import numpy as np
programasEnMapa = pd.DataFrame(l, columns=['x','y','Programa'])
programasEnMapa.insert(2, "Canal", np.array(programasDiaDf['EMPRESA']), True)
temasDiaNorm = [spanishNormalization(t) for t in tqdm(programasDiaDf['TIPO'])]
programasEnMapa.insert(2, "Tema", temasDiaNorm, True)
hora = [int(h[:2]) for h in programasDiaDf['HORA_INICIO']]
programasEnMapa.insert(2, "Hora inicio", np.array(programasDiaDf['HORA_INICIO']), True)
programasEnMapa.insert(2, "Hora", hora, True)

#Crear clase
sx = (np.array(programasEnMapa['x']) / 10).astype(int).astype(str)
sy = (np.array(programasEnMapa['y']) / 10).astype(int).astype(str)
clase = pd.Series(sx) +'-'+ pd.Series(sy)
programasEnMapa.insert(2, "Clase", clase, True)

In [177]:
clases = clase.unique()
tematicas = list()
for cl in clases:
  temas = programasEnMapa.loc[programasEnMapa['Clase']==cl, 'Tema']
  tematica = '-'.join((temas.value_counts()[:4]).index)
  tematicas.append(tematica)
clasesXtematicas = pd.DataFrame({'Clase':clases,'Tematica':tematicas})
clasesXtematicas

programasEnMapa = programasEnMapa.merge(clasesXtematicas, on='Clase', how='left')
programasEnMapa

,x,y,Clase,Hora,Hora inicio,Tema,Canal,Programa,Tematica
0,41,33,4-3,0,00:00:33,educativo,ASOCIACIÓN COMUNIDAD DE RESTAURACIÓN,HUELLAS DEL AMADO,promo-entrevistas-variedades-recreativo
1,11,25,1-2,0,00:30:15,educativo,ASOCIACIÓN COMUNIDAD DE RESTAURACIÓN,PLAN B,promo--variedades-ficción
2,49,21,4-2,1,01:00:14,predicación,ASOCIACIÓN COMUNIDAD DE RESTAURACIÓN,DESDE EL PÚLPITO,religioso-bumper-nacional valores-
3,44,34,4-3,2,02:30:15,predicación,ASOCIACIÓN COMUNIDAD DE RESTAURACIÓN,MENSAJE DE ESPERANZA,promo-entrevistas-variedades-recreativo
4,49,28,4-2,3,03:30:14,predicación,ASOCIACIÓN COMUNIDAD DE RESTAURACIÓN,EL PUEDE HACERLO DE NUEVO,religioso-bumper-nacional valores-
...,...,...,...,...,...,...,...,...,...
1420,44,31,4-3,20,20:30:00,opinion reportajes,Universidad del Valle,Charlas con Aguayo,promo-entrevistas-variedades-recreativo
1421,44,32,4-3,21,21:00:00,cultural opinión,Universidad del Valle,En Contacto,promo-entrevistas-variedades-recreativo
1422,49,2,4-0,22,22:00:00,cultural literatura,Universidad del Valle,Entre Libros arte y cultura,cultural-musical-promo-educativo
1423,25,35,2-3,22,22:30:00,informativo,Universidad del Valle,Noti 5,educativo-recreativo-informativo-


In [182]:
import altair as alt
from vega_datasets import data

source = programasEnMapa

input_dropdown = alt.binding_select(options=list(range(0,24)), name='Hora')
selection = alt.selection_single(fields=['Hora'], bind=input_dropdown)

alt.Chart(source, title="Programación para el día 31 de marzo de 2022").mark_circle(size=60,).encode(
    x='x',
    y='y',
    color=alt.Color('Tematica', scale=alt.Scale(scheme='tableau20')),
    tooltip=['Programa','Canal','Hora inicio']
).add_selection(
    selection
).transform_filter(
    selection
).interactive()

alt.Chart(...)